# Proposed Model
#### Read cropped images

In [1]:
import os
import numpy as np
from PIL import Image

species = ['Falciparum', 'Malariae', 'Ovale', 'Vivax']
filenames = []

for s in species:
    path = 'MP-IDB\\' + s + '\\cropped'
    filenames += [os.path.join(path, f) for f in os.listdir(path)]

img = []

for i in filenames:
    img.append(Image.open(i))

#### Image augmentation

In [2]:
import tensorflow as tf
import math

def augment(image):
    shape = tf.shape(image)
    dims_factor = tf.random.uniform([], 0.8, 1.0, dtype=tf.float32)
    height_dim  = tf.multiply(dims_factor, tf.cast(shape[0], tf.float32))
    width_dim   = tf.multiply(dims_factor, tf.cast(shape[1], tf.float32))
    image = tf.image.random_crop(image, [height_dim, width_dim, 3])
    rotate_prob = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    if rotate_prob > .75:
        image = tf.image.rot90(image, k=3)
    elif rotate_prob > .5:
        image = tf.image.rot90(image, k=2)
    elif rotate_prob > .25:
        image = tf.image.rot90(image, k=1)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2,)
    image = tf.image.random_contrast(image, 0.6, 1.4,)
    image = tf.image.random_hue(image, 0.0)
    image = tf.image.random_saturation(image, 0.5, 1.5,)
    return tf.image.resize(image, [64, 64])

#### Sampling

In [3]:
def sample(images, number):
    new_images = []
    for i in range(number // len(images)):
       for j in images:
           new_images.append(augment(j))
    for i in range(number % len(images)):
        new_images.append(augment(images[i]))
    return new_images

#### New Data

In [52]:
x_train = np.array(sample(img[:104], 3200) + sample(img[104:141], 3200) + sample(img[141:170], 3200) + sample(img[170:], 3200)).astype('float32') / 255
x_test = np.array(sample(img[:104], 80) + sample(img[104:141], 80) + sample(img[141:170], 80) + sample(img[170:], 80)).astype('float32') / 255

#### Labels

In [53]:
y_train = []
y_test = []
for i in range(4):
    for j in range(3200):
        y_train.append([0, 0, 0 ,0])
        y_train[-1][i] = 1
    for j in range(80):
        y_test.append([0, 0, 0 ,0])
        y_test[-1][i] = 1

y_train = np.array(y_train).astype('float32')
y_test = np.array(y_test).astype('float32')

### Autoencoder

#### Model

In [9]:
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Activation, Reshape

class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            Input((64, 64, 3)),
            Conv2D(16, (3, 3), padding = 'same'),
            Activation('relu'),
            MaxPooling2D((2, 2), padding = 'same'),
            Conv2D(8, (3, 3), padding = 'same'),
            Activation('relu'),
            MaxPooling2D((2, 2), padding = 'same'),
            Conv2D(4, (3, 3), padding = 'same'),
            Activation('relu'),
            MaxPooling2D((2, 2), padding = 'same')
        ])
        self.decoder = Sequential([
            Conv2DTranspose(4, (3, 3), padding = 'same'),
            Activation('relu'),
            UpSampling2D((2, 2)),
            Conv2DTranspose(8, (3, 3), padding = 'same'),
            Activation('relu'),
            UpSampling2D((2, 2)),
            Conv2DTranspose(16, (3, 3), padding = 'same'),
            Activation('relu'),
            UpSampling2D((2, 2)),
            Conv2DTranspose(3, (3, 3), padding = 'same'),
            Activation('relu'),
            Reshape((64, 64, 3))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder()
autoencoder.compile(optimizer = 'rmsprop', loss = MeanSquaredError())

#### Training

In [10]:
autoencoder.fit(x_train, x_train, epochs = 300, shuffle = True, validation_data = (x_test, x_test))

Epoch 1/300
200/200 [==============================] - 7s 28ms/step - loss: 0.0315 - val_loss: 0.0199
Epoch 2/300
200/200 [==============================] - 6s 28ms/step - loss: 0.0153 - val_loss: 0.0115
Epoch 3/300
200/200 [==============================] - 5s 27ms/step - loss: 0.0137 - val_loss: 0.0139
Epoch 4/300
200/200 [==============================] - 5s 26ms/step - loss: 0.0129 - val_loss: 0.0148
Epoch 5/300
200/200 [==============================] - 5s 27ms/step - loss: 0.0124 - val_loss: 0.0124
Epoch 6/300
200/200 [==============================] - 5s 27ms/step - loss: 0.0118 - val_loss: 0.0097
Epoch 7/300
200/200 [==============================] - 5s 26ms/step - loss: 0.0114 - val_loss: 0.0096
Epoch 8/300
200/200 [==============================] - 5s 27ms/step - loss: 0.0109 - val_loss: 0.0088
Epoch 9/300
200/200 [==============================] - 5s 27ms/step - loss: 0.0105 - val_loss: 0.0104
Epoch 10/300
200/200 [==============================] - 5s 27ms/step - loss: 0.010

#### Encode Images for Classification

In [54]:
x_train_encoded = autoencoder.encoder(x_train).numpy()
x_test_encoded = autoencoder.encoder(x_test).numpy()

### Classifier

#### Model

In [55]:
from tensorflow.keras.layers import Dense, Flatten

classifier = Sequential([
    Input((8, 8, 4)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dense(16),
    Activation('relu'),
    Dense(4),
    Activation('softmax')
])

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

#### Training

In [56]:
classifier.fit(x_train_encoded, y_train, epochs = 100, shuffle = True, validation_data = (x_test_encoded, y_test))

Epoch 1/100
400/400 [==============================] - 2s 2ms/step - loss: 0.9200 - val_loss: 0.8245
Epoch 2/100
400/400 [==============================] - 1s 2ms/step - loss: 0.7386 - val_loss: 0.7723
Epoch 3/100
400/400 [==============================] - 1s 2ms/step - loss: 0.6825 - val_loss: 0.7093
Epoch 4/100
400/400 [==============================] - 1s 2ms/step - loss: 0.6340 - val_loss: 0.7100
Epoch 5/100
400/400 [==============================] - 1s 2ms/step - loss: 0.6154 - val_loss: 0.6500
Epoch 6/100
400/400 [==============================] - 1s 2ms/step - loss: 0.5905 - val_loss: 0.6226
Epoch 7/100
400/400 [==============================] - 1s 2ms/step - loss: 0.5725 - val_loss: 0.6186
Epoch 8/100
400/400 [==============================] - 1s 2ms/step - loss: 0.5583 - val_loss: 0.6460
Epoch 9/100
400/400 [==============================] - 1s 2ms/step - loss: 0.5448 - val_loss: 0.6165
Epoch 10/100
400/400 [==============================] - 1s 2ms/step - loss: 0.5235 - val_lo

In [57]:
y_calculated = classifier(x_test_encoded).numpy()
true = [0, 0, 0, 0]
result = true.copy()

for i in range(4):
    for y in y_calculated[0 + i * 80 : 80 + i * 80]:
        pred = np.argmax(y)
        if pred == i:
            true[i] += 1
        result[pred] += 1

recall, precision = [], []
accuracy = 0

for i in range(4):
    recall.append(true[i] / 80)
    precision.append(true[i] / result[i])
    accuracy += true[i]

accuracy /= 320

In [58]:
print('Accuracy: ' + str(accuracy) + '\nRecall: ' + str(recall) + '\nPrecision: ' + str(precision))

Accuracy: 0.915625
Recall: [0.8625, 0.9, 0.9625, 0.9375]
Precision: [0.8961038961038961, 0.935064935064935, 0.9166666666666666, 0.9146341463414634]


### Merge models

In [59]:
model = Sequential([
    autoencoder.encoder,
    classifier
])

#### Save as .tflite file

In [64]:
model_lite = tf.lite.TFLiteConverter.from_keras_model(model).convert()

INFO:tensorflow:Assets written to: C:\Users\henglm\AppData\Local\Temp\tmpe7lsg25r\assets


INFO:tensorflow:Assets written to: C:\Users\henglm\AppData\Local\Temp\tmpe7lsg25r\assets


In [66]:
with open('classification.tflite', 'wb') as f:
    f.write(model_lite)